In [11]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2
import pandas as pd

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg

taxonomyDate = '20190420' #'20191017'
mappingDate = taxonomyDate[:6] if taxonomyDate.startswith('2019') else taxonomyDate #taxonomyDate #'201909' # where the {dataset}_mapping_to_GOD.pkl is

godLabelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_{}.tsv'.format(taxonomyDate)
godLabels = [l.rstrip().split('\t') for l in open(godLabelFile,'r').readlines()[1:]]
print (godLabels[-10:])

god_data = pd.read_csv(godLabelFile, delimiter='\t')
god_name_id_map = dict(zip(god_data['GOD_v1_name'], god_data['GOD_v1_id']))
god_classes = list(god_data['GOD_v1_name'])
print(god_classes[-10:])

def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
outValFolder = '/media/data/chnxi/GOD/json_annotations/val_{}'.format(taxonomyDate)

checkMkdir(outValFolder)
        
datasetList = ['HomeFurniture'] #['COCO_91', 'FashionV2', 'HomeFurniture',  'OpenImage', 'Object365'] # #, 


mappingFormat = '/media/data/chnxi/GOD/mappings/{}/dataset_nameid_to_GOD_nameid/{}_mapping_to_GOD.pkl'.format(mappingDate, '{}')

dataPath = '/media/data/chnxi/'
valSetFiles = {'COCO_91': dataPath + 'coco/annotations/instances_minival2014.json',
                'FashionV2': dataPath + 'FashionV2/json_annotations/val_annotations.json',
                'HomeFurniture': dataPath + 'HomeFurniture/json_annotations/val_annotations.json',
                'OpenImage': dataPath + 'OpenImage/json_annotations/validation_annotations.json',
                'Object365': dataPath + 'Object365/json_annotations/objects365_val_TrainCategs.json'
               }

[['visual_signs\\road_traffic_street_signs\\traffic_light', '570'], ['weapon\\bomb', '571'], ['weapon\\dagger', '572'], ['weapon\\handgun', '573'], ['weapon\\missile', '574'], ['weapon\\rifle', '575'], ['weapon\\rocket', '576'], ['weapon\\shotgun', '577'], ['weapon\\sword', '578'], ['wheel', '579']]
['visual_signs\\road_traffic_street_signs\\traffic_light', 'weapon\\bomb', 'weapon\\dagger', 'weapon\\handgun', 'weapon\\missile', 'weapon\\rifle', 'weapon\\rocket', 'weapon\\shotgun', 'weapon\\sword', 'wheel']


In [5]:
for cls, id in god_name_id_map.items():
    print('{}\t{}'.format(id, cls))

1	animal\amphibian\frog
2	animal\arthropod\arachnid\scorpion
3	animal\arthropod\arachnid\spider
4	animal\arthropod\crustacean\crab
5	animal\arthropod\crustacean\lobster
6	animal\arthropod\crustacean\shrimp
7	animal\arthropod\insect\ant
8	animal\arthropod\insect\bee
9	animal\arthropod\insect\butterfly
10	animal\arthropod\insect\caterpillar
11	animal\arthropod\insect\dragonfly
12	animal\arthropod\insect\ladybug
13	animal\arthropod\insect\tick
14	animal\arthropod\isopod
15	animal\arthropod\myriapod\centipede
16	animal\bird\blue_jay
17	animal\bird\canary
18	animal\bird\chicken
19	animal\bird\duck
20	animal\bird\eagle
21	animal\bird\falcon
22	animal\bird\goose
23	animal\bird\magpie
24	animal\bird\ostrich
25	animal\bird\owl
26	animal\bird\parrot
27	animal\bird\penguin
28	animal\bird\raven
29	animal\bird\sparrow
30	animal\bird\swan
31	animal\bird\turkey
32	animal\bird\woodpecker
33	animal\dinosaur
34	animal\fish\goldfish
35	animal\fish\ray
36	animal\jellyfish
37	animal\mammal\alpaca
38	animal

In [6]:
dataset = 'HomeFurniture'
mapFile = mappingFormat.format(dataset)
mapping = pkl.load(open(mapFile,'rb'))
print(mapping.keys())
print(mapping['name_to_god_name'])
# print (mapping['name_to_god_name'])

dict_keys(['name_to_god_name', 'id_map', 'name_to_god_id'])
{'toilet': 'home_or_office_furnishing_or_decor\\toilet', 'ladder_or_stepstool': 'home_or_office_furnishing_or_decor\\ladder_or_step_stool', 'tv_monitor': 'electronics\\home_entertainment\\tv', 'bed_linens': 'home_or_office_furnishing_or_decor\\bedlinen', 'crib': 'home_or_office_furnishing_or_decor\\bed\\crib', 'throwpillow': 'home_or_office_furnishing_or_decor\\pillow_or_cushion\\cushion', 'nightstand': 'home_or_office_furnishing_or_decor\\storage\\nightstand', 'oven': 'electronics\\home_appliances\\wall_oven', 'mirror': 'home_or_office_furnishing_or_decor\\mirror', 'comforter': 'home_or_office_furnishing_or_decor\\comforter', 'chair': 'home_or_office_furnishing_or_decor\\seating\\chair', 'kitchen_faucet': 'home_or_office_furnishing_or_decor\\faucet', 'shelving': 'home_or_office_furnishing_or_decor\\storage\\shelving', 'rug_or_mat': 'home_or_office_furnishing_or_decor\\rug_or_mat', 'microwave': 'electronics\\home_appliances\\m

In [9]:
sorted(mapping['name_to_god_name'].keys())

['armchair',
 'basket',
 'bathtub',
 'bed',
 'bed_linens',
 'bench',
 'cabinet',
 'candle_or_candle_holder',
 'ceiling_fan',
 'ceiling_light',
 'chair',
 'comforter',
 'cooktop',
 'crib',
 'curtain',
 'desk_or_table',
 'dishwasher',
 'drawer_chest',
 'fireplace',
 'floor_lamp',
 'frame_and_picture',
 'kitchen_faucet',
 'kitchen_island',
 'kitchen_sink',
 'ladder_or_stepstool',
 'microwave',
 'mirror',
 'nightstand',
 'oven',
 'paper_decoration',
 'pitcher',
 'plant_and_pot',
 'range_hood',
 'refrigerator',
 'rug_or_mat',
 'shelving',
 'shower',
 'sofa',
 'table_lamp',
 'throwpillow',
 'toilet',
 'towel',
 'toy_organizer',
 'tv_monitor',
 'vase',
 'wardrobe',
 'washer_and_dryer']

In [ ]:
updateAnno = True
for dataset in datasetList:
    print ("dataset = {}".format(dataset))
    outFileName = osp.join(outValFolder, 'GOD_{}_val_annotations.json'.format(dataset))
    print ("outFileName = {}".format(outFileName))
    mapFile = mappingFormat.format(dataset)
    mapping = pkl.load(open(mapFile,'rb'))
    #print (mapping.keys())
    id_map = mapping['id_map']
    name_map = mapping['name_to_god_name']
    
    annoFile = valSetFiles[dataset]
    annoData = json.load(open(annoFile,'r'))
    #print (anno.keys())
    
    
    godCateg = []
    supercat = 'GOD_{}'.format(dataset)
    for god_categ in godLabels:
        cls = god_categ[1] #[0]
        id = int(god_categ[0])
        cats = {'supercategory':supercat,
                'id':id,
                'name':cls}
        godCateg.append(cats) 
        
        
    subsetCateg = []
    for org_cat in annoData['categories']:
        org_id = org_cat['id']
        if org_id in id_map:
            god_id = id_map[org_id]
            org_name = org_cat['name']
            org_name = org_name.strip().lower()
            if dataset == 'OpenImage':
                org_name = org_name.lower().replace('_',' ')
            cats = {'supercategory':supercat,
                    'id':god_id,
                    'name':name_map[org_name]}
            subsetCateg.append(cats)
    
    
    
    #################################################################################
#     for ix, categ in enumerate(annoData['categories']):
#         src_id = categ['id']
#         if src_id in id_map:
#             categ['id'] = id_map[src_id]
# #             print ("{}==>{}".format(src_id, id_map[src_id]))
# #             print (categ)
#             newCateg.append(categ)
#         else:
#             print ("{} not in GOD".format(categ['name']))
    #################################################################################
    
    print(godCateg[:10])
    print (subsetCateg[:10])
    annoData['categories'] = godCateg
    annoData['subset_categories'] = subsetCateg
    
    #################################################################################
    if osp.exists(outFileName) and not updateAnno:
        godAnnoData = json.load(open(outFileName,'r'))
        annoData['annotations'] = godAnnoData['annotations']
    else: # if not exists(outFileName) or  updateAnno
        newAnnos = []
        for ox, obj in enumerate(annoData['annotations']):
            if obj['category_id'] in id_map:
                src_id = obj['category_id']
                god_id = id_map[obj['category_id']]
                obj['category_id'] = god_id
                newAnnos.append(obj)
                if (ox + 1) % 10000 == 0:
                    print ("{}==>{}".format(src_id, god_id))
    #                 print (obj)
        annoData['annotations'] = newAnnos
    print ("saving to {}".format(outFileName))
    with open(outFileName,'w') as fout:
        json.dump(annoData, fout)
    print ("Done")
    